In [1]:
import pandas as pd
# reading csv files
colTitles = ['age','workclass','fnlwgt','education','education-num'\
             ,'marital-status','occupation','relationship','race','sex'\
             ,'capital-gain','capital-loss','hours-per-week','native-country','income']
data =  pd.read_csv('adult.data', sep=",",names=colTitles)
data = data.drop(columns=['fnlwgt','education-num','relationship','capital-gain','capital-loss'])
print(data)

       age          workclass  ...  native-country  income
0       39          State-gov  ...   United-States   <=50K
1       50   Self-emp-not-inc  ...   United-States   <=50K
2       38            Private  ...   United-States   <=50K
3       53            Private  ...   United-States   <=50K
4       28            Private  ...            Cuba   <=50K
...    ...                ...  ...             ...     ...
32556   27            Private  ...   United-States   <=50K
32557   40            Private  ...   United-States    >50K
32558   58            Private  ...   United-States   <=50K
32559   22            Private  ...   United-States   <=50K
32560   52       Self-emp-inc  ...   United-States    >50K

[32561 rows x 10 columns]


In [2]:
rows = data.shape[0]
for row in range(rows):
  # age: 5 years per class
  data.iloc[row,0] = data.iloc[row,0]//5
  # hours-per-week: 10 hrs per class
  data.iloc[row,7] = data.iloc[row,7]//10
  # income: 1: >50k, 0:<= 50k
  if data.iloc[row,9] == ' <=50K':
    data.iloc[row,9] = 0
  else:
    data.iloc[row,9] = 1
print(data)

       age          workclass  ...  native-country income
0        7          State-gov  ...   United-States      0
1       10   Self-emp-not-inc  ...   United-States      0
2        7            Private  ...   United-States      0
3       10            Private  ...   United-States      0
4        5            Private  ...            Cuba      0
...    ...                ...  ...             ...    ...
32556    5            Private  ...   United-States      0
32557    8            Private  ...   United-States      1
32558   11            Private  ...   United-States      0
32559    4            Private  ...   United-States      0
32560   10       Self-emp-inc  ...   United-States      1

[32561 rows x 10 columns]


In [3]:
! pip install apyori
import numpy as np
import matplotlib.pyplot as plt
from apyori import apriori


  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=848e32f361c039bda922265796e6bda757bf4e6f880dbf68a17d0f2cadcd099e
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [4]:
rows = data.shape[0]
cols = data.shape[1]
attributes = list(data.columns)
dataList = []
for row in range(rows):
  oneRow = []
  for col in range(cols):
    oneRow.append(attributes[col]+':'+str(data.iloc[row,col]))
  dataList.append(oneRow)
print(dataList[0:10])

[['age:7', 'workclass: State-gov', 'education: Bachelors', 'marital-status: Never-married', 'occupation: Adm-clerical', 'race: White', 'sex: Male', 'hours-per-week:4', 'native-country: United-States', 'income:0'], ['age:10', 'workclass: Self-emp-not-inc', 'education: Bachelors', 'marital-status: Married-civ-spouse', 'occupation: Exec-managerial', 'race: White', 'sex: Male', 'hours-per-week:1', 'native-country: United-States', 'income:0'], ['age:7', 'workclass: Private', 'education: HS-grad', 'marital-status: Divorced', 'occupation: Handlers-cleaners', 'race: White', 'sex: Male', 'hours-per-week:4', 'native-country: United-States', 'income:0'], ['age:10', 'workclass: Private', 'education: 11th', 'marital-status: Married-civ-spouse', 'occupation: Handlers-cleaners', 'race: Black', 'sex: Male', 'hours-per-week:4', 'native-country: United-States', 'income:0'], ['age:5', 'workclass: Private', 'education: Bachelors', 'marital-status: Married-civ-spouse', 'occupation: Prof-specialty', 'race: 

In [5]:
rules = apriori(transactions = dataList, min_support = 0.02, min_confidence = 0.2, min_lift = 3, min_length = 3, max_length = 3)
print(rules)
results = list(rules)
results

def inspect(results):
    lhs         = [tuple(result[2][0][0]) for result in results]
    rhs         = [tuple(result[2][0][1]) for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
resultsinDataFrame

<generator object apriori at 0x7f962404add0>


,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,"(education: Masters,)","(occupation: Prof-specialty,)",0.025921,0.489843,3.852606
1,"(occupation: ?,)","(workclass: ?,)",0.056386,0.996202,17.667390
2,"(age:3,)","(income:0, marital-status: Never-married)",0.049507,0.972842,3.107999
3,"(age:3,)","(native-country: United-States, marital-status...",0.046589,0.915510,3.112007
4,"(age:3,)","(race: White, marital-status: Never-married)",0.043703,0.858781,3.193190
5,"(age:3,)","(sex: Female, marital-status: Never-married)",0.024016,0.471937,3.223568
6,"(age:4,)","(education: Some-college, marital-status: Neve...",0.043058,0.358293,3.977625
7,"(marital-status: Married-civ-spouse, education...","(income:1,)",0.023771,0.771685,3.204545
8,"(education: Masters,)","(occupation: Prof-specialty, native-country: U...",0.022972,0.434127,3.827672
9,"(education: Masters,)","(occupation: Prof-specialty, race: White)",0.023034,0.435287,3.882057
